In [38]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random

In [39]:
# Intrinsic Curiosity Module (ICM)
class StateEncoder(nn.Module):
    def __init__(self, state_dim, encoding_dim):
        super(StateEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(state_dim, 256),
            nn.ReLU(),
            nn.Linear(256, encoding_dim),
        )

    def forward(self, state):
        return self.encoder(state)

class InverseModel(nn.Module):
    def __init__(self, encoding_dim, action_dim):
        super(InverseModel, self).__init__()
        self.inverse_model = nn.Sequential(
            nn.Linear(encoding_dim * 2, 256),
            nn.ReLU(),
            nn.Linear(256, action_dim),
        )

    def forward(self, state_encoding, next_state_encoding):
        input_tensor = torch.cat([state_encoding, next_state_encoding], dim=0)
        return self.inverse_model(input_tensor)

class ForwardModel(nn.Module):
    def __init__(self, encoding_dim, action_dim, stack_size=4):
        super(ForwardModel, self).__init__()
        self.stack_size = stack_size
        self.forward_model = nn.Sequential(
            nn.Linear(encoding_dim * stack_size + action_dim, 256),
            nn.ReLU(),
            nn.Linear(256, encoding_dim),
        )

    def forward(self, state_encoding_stack, action):
        input_tensor = torch.cat([state_encoding_stack.view(-1, self.stack_size * state_encoding_stack.shape[-1]), action], dim=1)
        return self.forward_model(input_tensor)

class ICM(nn.Module):
    def __init__(self, state_dim, action_dim, encoding_dim, stack_size=4):
        super(ICM, self).__init__()
        self.state_encoder = StateEncoder(state_dim, encoding_dim)
        self.inverse_model = InverseModel(encoding_dim, action_dim)
        self.forward_model = ForwardModel(encoding_dim, action_dim, stack_size)

    def forward(self, state, next_state, action):
        state_encoding = self.state_encoder(state)
        next_state_encoding = self.state_encoder(next_state)

        inverse_loss = nn.MSELoss()(self.inverse_model(state_encoding, next_state_encoding), action)

        state_encoding_stack = torch.cat([state_encoding.unsqueeze(1)] * self.forward_model.stack_size, dim=1)
        forward_loss = nn.MSELoss()(self.forward_model(state_encoding_stack, action), next_state_encoding)

        return inverse_loss, forward_loss


In [40]:
# Deep Q-Network (DQN)
class DQN(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, action_dim)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)


In [41]:

# Replay Buffer
class ReplayBuffer:
    def __init__(self, buffer_size):
        self.buffer = deque(maxlen=buffer_size)

    def append(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = map(np.stack, zip(*batch))
        return state, action, reward, next_state, done


In [42]:

# Train Agent
def train_agent(env, episodes, batch_size, buffer_size, gamma, epsilon, epsilon_decay):
    state_dim = env.observation_space.shape[0]
    # action_dim = env.action_space.shape[0]
    action_dim = 3
    encoding_dim = 64
    stack_size = 4

    dqn = DQN(state_dim, action_dim).to(device)
    icm = ICM(state_dim, action_dim, encoding_dim, stack_size).to(device)
    dqn_optimizer = optim.Adam(dqn.parameters())
    icm_optimizer = optim.Adam(icm.parameters())
    replay_buffer = ReplayBuffer(buffer_size)

    for episode in range(episodes):
        state = env.reset()
        episode_reward = 0
        done = False

        while not done:
            # Choose action
            if np.random.rand() < epsilon:
                action = env.action_space.sample()
            else:
                state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
                q_values = dqn(state_tensor)
                action = torch.argmax(q_values).item()

            # Take action and observe next state
            print(action)
            next_state, reward, done = env.step(action)[:3]

            print(next_state, reward, done)
            print (state)
            
            #convert state from tuple to pytorch tensor>
            state = np.array(state[0])
            print(state.shape)
            
            # Compute intrinsic reward
            state_tensor = torch.tensor(state, dtype=torch.float32, device=device).reshape(1, -1)
            next_state_tensor = torch.tensor(next_state, dtype=torch.float32, device=device).reshape(1, -1)
            action_tensor = torch.tensor([action], dtype=torch.float32, device=device)

            inverse_loss, forward_loss = icm(state_tensor.squeeze(0), next_state_tensor.squeeze(0), action_tensor)
            intrinsic_reward = forward_loss.item()

            # Add experience to replay buffer
            replay_buffer.append(state, action, reward + intrinsic_reward, next_state, done)

            # Sample from replay buffer and update networks
            if len(replay_buffer.buffer) >= batch_size:
                states, actions, rewards, next_states, dones = replay_buffer.sample(batch_size)

                states_tensor = torch.tensor(states, dtype=torch.float32, device=device)
                actions_tensor = torch.tensor(actions, dtype=torch.long, device=device).unsqueeze(1)
                rewards_tensor = torch.tensor(rewards, dtype=torch.float32, device=device).unsqueeze(1)
                next_states_tensor = torch.tensor(next_states, dtype=torch.float32, device=device)
                dones_tensor = torch.tensor(dones, dtype=torch.float32, device=device).unsqueeze(1)

                # Update DQN
                q_values = dqn(states_tensor).gather(1, actions_tensor)
                next_q_values = dqn(next_states_tensor).max(1)[0].detach()
                expected_q_values = rewards_tensor + gamma * next_q_values * (1 - dones_tensor)
                dqn_loss = nn.MSELoss()(q_values, expected_q_values)

                dqn_optimizer.zero_grad()
                dqn_loss.backward()
                dqn_optimizer.step()

                # Update ICM
                icm_optimizer.zero_grad()
                inverse_loss, forward_loss = icm(states_tensor, next_states_tensor, actions_tensor)
                icm_loss = inverse_loss + forward_loss
                icm_loss.backward()
                icm_optimizer.step()

            state = next_state
            episode_reward += reward

        epsilon *= epsilon_decay
        print(f"Episode {episode + 1}, Reward: {episode_reward}, Epsilon: {epsilon}")

    env.close()


In [44]:

if __name__ == "__main__":
    env = gym.make("Hopper-v2")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    episodes = 1000
    batch_size = 64
    buffer_size = 100000
    gamma = 0.99
    epsilon = 1.0
    epsilon_decay = 0.995

    train_agent(env, episodes, batch_size, buffer_size, gamma, epsilon, epsilon_decay)

[ 0.5689214  -0.3068337  -0.31598258]
[ 1.25290218e+00  1.57711218e-03 -1.15017681e-03 -2.00021029e-03
 -5.16223835e-03 -1.54648040e-03 -6.70833983e-02  5.18844074e-01
  8.73329144e-01 -4.60344515e-01 -4.47067941e-01] 0.9995998568170228 False
(array([ 1.25317181e+00, -5.10535800e-04, -4.63249702e-03, -1.56694872e-04,
       -3.36991884e-03,  1.78174941e-03, -4.99624155e-04,  2.37616947e-03,
       -3.87319959e-03,  4.36671896e-05, -5.09108664e-04]), {})
(11,)


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 16 but got size 1 for tensor number 1 in the list.